In [390]:
# Python calculation for the example and exercise from Lecture 4 notes
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
import scipy.interpolate
import statistics
from statistics import NormalDist

In [392]:
hist_data = pd.read_excel("hist_data.xlsm")

In [394]:
#daily movement from swap rate
SOFR = pd.read_excel("hist_data.xlsm",sheet_name=0)
SOFR_new=SOFR.set_index("T")
SOFR_new=SOFR_new.drop(["Tenor"],axis=1)
SOFR_T=SOFR_new.T
SOFR_T.index=pd.to_datetime(SOFR_T.index).date

In [396]:
#SOFR Absolute Variable
SOFR_val=(SOFR_T.diff()*10000).dropna() #bps
corr_sofr=SOFR_val.corr().values
cov_sofr=SOFR_val.cov().values
SOFR_M=SOFR_val.mean().values
SOFR_SD=SOFR_val.std().values

In [416]:
# Monte Carlo VaR - SOFR
n_SOFR = SOFR_val.shape[1] #this is the number of tenors --> therefore number of risks
factor_loadings_sofr = np.linalg.cholesky(corr_sofr)
np.random.seed(29)
uniforms_sofr = np.random.uniform(size=(100000,n_SOFR))
snorms_sofr = [ [NormalDist().inv_cdf(k) for k in g]  for g in uniforms_sofr]
snorms_correlated_sofr = np.dot(snorms_sofr, factor_loadings_sofr.transpose())

In [417]:
abs_chng_sofr_mc = (snorms_correlated_sofr * SOFR_SD) + SOFR_M

sofr_curve_t0 = SOFR_T.iloc[-1] #latest value
Tenor_of_SOFR_new = range(1,11) #1Y to 10Y
swap_zero_rates_t0 = sofr_curve_t0[Tenor_of_SOFR_new].values

swap_pnl_1d_full_mc = []
swap_pnl_1d_sens_mc = []
for delta in abs_chng_sofr_mc:
    swap_zero_rates_delta = delta[Tenor_of_SOFR_new]/10000 # this returns the delta for the respective tenors
    swap_zero_rates_t1 = swap_zero_rates_t0 + swap_zero_rates_delta
    pnl_full = swap_pnl_1d_full(swap_zero_rates_t0,swap_zero_rates_t1,Tenor_of_SOFR_new)
    pnl_sens = swap_pnl_1d_sens(swap_zero_rates_t0,swap_zero_rates_delta,Tenor_of_SOFR_new)
    swap_pnl_1d_full_mc.append(pnl_full)
    swap_pnl_1d_sens_mc.append(pnl_sens)

print(np.mean(swap_pnl_1d_full_mc))
print(np.mean(swap_pnl_1d_sens_mc))

12621.44370645612
15148.280518389363


In [418]:
#SOFR
def get_discount_curve(zero_rates,tenors):
    Z = np.array(zero_rates)
    T = np.array(tenors)
    return np.exp(-Z*T)


def get_forward_curve(zero_rates,tenors):
    DF = get_discount_curve(zero_rates, tenors)
    DF_start = np.concatenate([[1], DF[:-1]]) # first DF=1. we dont consider forward swap here
    DF_end = DF
    F = (DF_start - DF_end) / DF_end
    return F
    

def get_payer_swap_pv(zero_rates,tenors,swap_rate=0.042, notional=100000000):
    DF = get_discount_curve(zero_rates, tenors)
    F = get_forward_curve(zero_rates, tenors)
    pv_fix = swap_rate*sum(DF)
    pv_flt = sum(F*DF)
    return notional*(pv_flt - pv_fix)


def swap_pnl_1d_full(zero_rates_t0,zero_rates_t1,tenors):
    pv_t0 = get_payer_swap_pv(zero_rates_t0, tenors)
    pv_t1 = get_payer_swap_pv(zero_rates_t1, tenors)
    return pv_t1 - pv_t0


def swap_pnl_1d_sens(zero_rates_t0,zero_rates_chng,tenors,swap_rate=0.042,notional=100000000):    
    zero_rates_chng = np.array(zero_rates_chng)
    tenors = np.array(tenors)
    DF_t0 = get_discount_curve(zero_rates_t0, tenors)
    W = notional * swap_rate * tenors * DF_t0  # weight of risk factors in PnL sensitivity
    W[-1] = notional * (1+swap_rate) * tenors[-1] * DF_t0[-1]
    return W @ zero_rates_chng

In [420]:
AAPL = pd.read_excel("/Users/jiayu/Downloads/hist_data.xlsm",sheet_name=1).set_index("Date")
AAPL.index=pd.to_datetime(AAPL.index,dayfirst=True).date
AAPL=AAPL.sort_index()

MSFT = pd.read_excel("/Users/jiayu/Downloads/hist_data.xlsm",sheet_name=2).set_index("Date")
MSFT.index=pd.to_datetime(MSFT.index,dayfirst=True).date
MSFT=MSFT.sort_index()

F= pd.read_excel("/Users/jiayu/Downloads/hist_data.xlsm",sheet_name=3).set_index("Date")
F.index=pd.to_datetime(F.index,dayfirst=True).date
F=F.sort_index()

BAC = pd.read_excel("/Users/jiayu/Downloads/hist_data.xlsm",sheet_name=4).set_index("Date")
BAC.index=pd.to_datetime(BAC.index,dayfirst=True).date
BAC=BAC.sort_index()


In [421]:
closing_all = pd.DataFrame({ 'AAPL': pd.Series(AAPL["Adj Close"]).pct_change().dropna(),'BAC': pd.Series(BAC["Adj Close"]).pct_change().dropna(),'F': pd.Series(F["Adj Close"]).pct_change().dropna(),'MSFT': pd.Series(MSFT["Adj Close"]).pct_change().dropna()})

In [422]:
corr_mat=closing_all.corr()
cov_mat=closing_all.cov()

AAPL_M=closing_all.loc[:,"AAPL"].mean()
AAPL_SD=closing_all.loc[:,"AAPL"].std()

MSFT_M=closing_all.loc[:,"MSFT"].mean()
MSFT_SD=closing_all.loc[:,"MSFT"].std()

F_M=closing_all.loc[:,"F"].mean()
F_SD=closing_all.loc[:,"F"].std()

BAC_M=closing_all.loc[:,"BAC"].mean()
BAC_SD=closing_all.loc[:,"BAC"].std()


mean_vec=np.array([AAPL_M,MSFT_M,F_M,BAC_M])
Last_Price=closing_all.iloc[-1]

In [423]:
# full revaluation 1d pnl evaluation
def pnl1d_full(aapl_return, bac_return, f_return, msft_return):
    return 100000*(Last_Price[0]*((1+aapl_return)-1) + Last_Price[3]*((1+msft_return)-1)+Last_Price[2]*((1+f_return)-1)+Last_Price[1]*((1+bac_return)-1))

# sensitivity based 1d pnl evaluation 
def pnl1d_sen(aapl_return, bac_return, f_return, msft_return):
    return  100000*(Last_Price[0]*aapl_return+Last_Price[3]*msft_return+Last_Price[2]*f_return+ Last_Price[1]*bac_return)


In [424]:
# Monte Carlo VaR - Stocks
n_mc = 100000
factor_loadings = np.linalg.cholesky(corr_mat)
np.random.seed(329)
uniforms = np.random.uniform(size=(n_mc,4))
snorms = [ [NormalDist().inv_cdf(u) for u in r]  for r in uniforms]
snorms_correlated = np.dot(snorms, factor_loadings.transpose())
return1d_sample = [[ AAPL_M + AAPL_SD * z[0],   MSFT_M + MSFT_SD * z[1], F_M + F_SD * z[2], BAC_M + BAC_SD * z[3]]  for z in snorms_correlated]
pnl1d_full_sample =  np.array([pnl1d_full(s[0], s[1], s[2],s[3])  for s in return1d_sample])+ np.array(swap_pnl_1d_full_mc)
var1d_full_mc = np.abs(np.percentile(pnl1d_full_sample, 5))
pnl1d_sen_sample = np.array([ pnl1d_sen(s[0], s[1], s[2],s[3])  for s in return1d_sample])+ np.array(swap_pnl_1d_sens_mc)
var1d_sen_mc = np.abs(np.percentile(pnl1d_sen_sample, 5))
print("")
print("")
print("============================================================================================================================")
print("Monte Carlo VaR:")
print(f"VaR [1d, 95%], Full Revaluation: {var1d_full_mc:,.0f}") 
print("")
print(f"VaR [1d, 95%], Sensitivity: {var1d_sen_mc:,.0f}") 
print("============================================================================================================================")


/var/folders/mb/3crcdgmn079fvqs204k59lg40000gn/T/ipykernel_33882/3658623327.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return 100000*(Last_Price[0]*((1+aapl_return)-1) + Last_Price[3]*((1+msft_return)-1)+Last_Price[2]*((1+f_return)-1)+Last_Price[1]*((1+bac_return)-1))
/var/folders/mb/3crcdgmn079fvqs204k59lg40000gn/T/ipykernel_33882/3658623327.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return  100000*(Last_Price[0]*aapl_return+Last_Price[3]*msft_return+Last_Price[2]*f_return+ Last_Price[1]*bac_return)




Monte Carlo VaR:
VaR [1d, 95%], Full Revaluation: 1,042,078

VaR [1d, 95%], Sensitivity: 1,035,052
